# 0. Install and Import dependencies

We will use the most advanced environment for RL-trading task - DI-engine/gym-anytrade. It's a part of DI-engine RL framework, which contains a lot of environments inside.

**Important notes: tensorflow==1.15.0 works only with Python 3.7 and below**

In [1]:
!git clone https://github.com/opendilab/DI-engine.git
%pip install DI-engine/.
!rm -rf DI-engine

%pip install gym stable-baselines finta tensorflow-gpu==1.15.0 tensorflow==1.15.0 

Processing ./DI-engine
  Preparing metadata (setup.py) ... done
  Using cached gym-0.25.2-py3-none-any.whl
  Using cached torch-1.12.1-cp37-cp37m-manylinux1_x86_64.whl (776.3 MB)
  Using cached numpy-1.21.6-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (15.7 MB)
  Using cached pandas-1.3.5-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (11.3 MB)
  Using cached tensorboardX-2.5.1-py2.py3-none-any.whl (125 kB)
  Using cached requests-2.28.1-py3-none-any.whl (62 kB)
  Using cached PyYAML-6.0-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (596 kB)
  Using cached easydict-1.9-py3-none-any.whl
  Using cached protobuf-4.21.9-cp37-abi3-manylinux2014_x86_64.whl (408 kB)
  Using cached yapf-0.29.0-py2.py3-none-any.whl (185 kB)
  Using cached Flask-1.1.4-py2.py3-none-any.whl (94 kB)
  Using cached tqdm-4.64.1-py2.py3-none-any.whl (78 kB)
  Using cached lz4-4.0.2-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.2 

Installation Check

In [2]:
import gym

# Stable baselines - rl stuff.
from stable_baselines.common.vec_env import DummyVecEnv
from stable_baselines import A2C

# Processing libraries.
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

# Finta is needed to calculate Market Custom indicators.
from finta import TA

import ding
print(ding.__version__)

/home/ravil/Desktop/Innopolis/Classes/3_sem/RL/RL-trading-agent/venv/lib/python3.7/site-packages/tensorflow_core/python/pywrap_tensorflow_internal.py:15: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
/home/ravil/Desktop/Innopolis/Classes/3_sem/RL/RL-trading-agent/venv/lib/python3.7/site-packages/tensorflow_core/python/framework/dtypes.py:597: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  np.object,
/home/ravil/Desktop/Innopolis/Classes/3_sem/RL/RL-trading-agent/venv/lib/python3.7/site-packages/tensorflow_core/python/framework/dtypes.py:605: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning,

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



/home/ravil/Desktop/Innopolis/Classes/3_sem/RL/RL-trading-agent/venv/lib/python3.7/site-packages/stable_baselines/__init__.py:33: UserWarning: stable-baselines is in maintenance mode, please use [Stable-Baselines3 (SB3)](https://github.com/DLR-RM/stable-baselines3) for an up-to-date version. You can find a [migration guide](https://stable-baselines3.readthedocs.io/en/master/guide/migration.html) in SB3 documentation.
  "stable-baselines is in maintenance mode, please use [Stable-Baselines3 (SB3)](https://github.com/DLR-RM/stable-baselines3) for an up-to-date version. You can find a [migration guide](https://stable-baselines3.readthedocs.io/en/master/guide/migration.html) in SB3 documentation."


v0.4.4


# 1. Data preprocessing

In [ ]:
# Download some data, if you need

In [7]:
import os

csv_name = 'STOCKS_GOOGL.csv'

# check the ding installation folder
ding_folder = ding.__file__.split('/ding')[0]
data_folder = os.path.join(ding_folder, 'dizoo/gym_anytrading/envs/data')  

if os.path.exists(data_folder) == 0:
    os.mkdir(data_folder)

# move data to correct folder
os.replace(f"{csv_name}", f"{data_folder}/example_data.csv")

In [ ]:
# Enhance with custom indicators using FINTA

# 2. Create new environment


In [22]:
import numpy as np
from easydict import EasyDict
from dizoo.gym_anytrading.envs import StocksEnv

new_env = StocksEnv(EasyDict({"env_id": 'stocks-v0', "eps_length": 300,\
            "window_size": 20, "train_range": None, "test_range": None, "stocks_data_filename": 'example_data'}))

In [23]:
new_env.reset()

array([-5.7169259e-01, -5.3066853e-03,  2.0483774e-01, -5.7478660e-01,
       -3.0940038e-03, -1.8333234e-02, -5.7090110e-01,  3.8854936e-03,
       -4.9585402e-01, -5.6197864e-01,  8.9224614e-03, -1.0771014e-01,
       -5.5285829e-01,  9.1203311e-03,  6.1099446e-01, -5.2774584e-01,
        2.5112454e-02,  7.5697792e-01, -5.5363184e-01, -2.5885954e-02,
        6.3584346e-01, -5.4810917e-01,  5.5226553e-03,  1.1722134e+00,
       -5.4058993e-01,  7.5192587e-03,  1.2251406e-01, -5.5667186e-01,
       -1.6081952e-02,  3.0318490e-01, -5.5782324e-01, -1.1513666e-03,
        2.2990379e-01, -5.4046398e-01,  1.7359236e-02,  1.0243952e-01,
       -5.3790951e-01,  2.5544616e-03,  2.7565926e-01, -5.2893311e-01,
        8.9764278e-03,  2.1033564e-01, -5.0139219e-01,  2.7540883e-02,
        6.4409029e-01, -4.7063130e-01,  3.0760920e-02,  1.5608275e+00,
       -4.8361924e-01, -1.2987948e-02,  7.1447778e-01, -4.8142466e-01,
        2.1945811e-03,  1.2164597e-01, -4.8336741e-01, -1.9427451e-03,
      

In [24]:
print(new_env.action_space)

Discrete(5)


### Indicators can be added by overriding _process_data method.

# 3. Build Environment and Train

In [27]:
env = DummyVecEnv([lambda: new_env])
model = A2C('MlpLstmPolicy', env, verbose=1) 
model.learn(total_timesteps=1000000)

AttributeError: module 'gym.logger' has no attribute 'MIN_LEVEL'

# Next step - Create a training and evaluation pipelines.

# Usefull links:
1) DI-engine/gym-anytading: https://github.com/opendilab/DI-engine/tree/main/dizoo/gym_anytrading
2) Original gym-anytrading's use example: https://github.com/AminHP/gym-anytrading/blob/master/examples/a2c_quantstats.ipynb
3) Adding custom signals to original gym-anytrading: https://github.com/nicknochnack/Reinforcement-Learning-for-Trading-Custom-Signals/blob/main/Custom%20Signals.ipynb